In [40]:
import os
import pandas as pd
from glob import glob

In [2]:
def process_zufang_data(df):
    # Function to extract information from title
    def extract_details(title):
        if pd.isna(title):
            return pd.Series([None, None, None, None])
        parts = title.split('·')
        if len(parts) < 2:
            return pd.Series([None, None, None, None])
        
        rent_type = parts[0]
        rest = parts[1].split(' ')
        
        community = rest[0]
        total_room = None
        orientate = None
        
        if len(rest) > 1:
            total_room = rest[1]
        if len(rest) > 2:
            orientate = rest[2]
        
        return pd.Series([rent_type, community, total_room, orientate])

    # Function to process and extract the minimum price if it's a range
    def process_price(price):
        # Remove '元/月'
        price = price.replace('元/月', '')
        # Check if price contains a range
        if '-' in price:
            # Split the string by '-' and convert each part to integer
            prices = list(map(int, price.split('-')))
            # Return the minimum value
            return min(prices)
        else:
            # Return the price as integer
            return int(price)

    # Apply the functions to extract details and process price
    df[['rent_type', 'community', 'total_room', 'orientate']] = df['title'].apply(extract_details)
    df['price'] = df['price'].apply(process_price)

    return df

In [3]:
def process_chengjiao_data(df):
    # 1. Separate the 'title' into 'community', 'total_room', 'area'
    df[['community', 'total_room', 'area']] = df['title'].str.extract(r'^(.*?)\s(.*?)\s(.*?)平米$')

    # 2. Remove '元/平' from the 'unit_price'
    df['unit_price'] = df['unit_price'].str.replace('元/平', '', regex=False)

    # 3. Remove '万' from the 'total_price'
    df['total_price'] = df['total_price'].str.replace('万', '', regex=False)

    # 4. Remove '挂牌' and '万' from 'list_amount'
    df['list_amount'] = df['list_amount'].str.replace('挂牌', '').str.replace('万', '', regex=False)

    # 5. Remove '成交周期' and '天' from 'transaction_day'
    df['transaction_day'] = df['transaction_day'].str.replace('成交周期', '').str.replace('天', '', regex=False)

    # Convert cleaned columns to appropriate types
    df['unit_price'] = pd.to_numeric(df['unit_price'], errors='coerce')
    df['total_price'] = pd.to_numeric(df['total_price'], errors='coerce')
    df['list_amount'] = pd.to_numeric(df['list_amount'], errors='coerce')
    df['transaction_day'] = pd.to_numeric(df['transaction_day'], errors='coerce')

    return df

In [8]:
# Define the district names
district_names = ['jingan', 'xuhui', 'huangpu', 'changning', 'putuo', 'pudong', 'hongkou', 'yangpu']

# Base paths for the data directories
zufang_base_path = os.path.join('/Users','toshiro','repos','LianJia', 'zufang', 'data')
chengjiao_base_path = os.path.join('/Users','toshiro','repos','LianJia', 'chengjiao', 'data')

# Loop through each district
for district in district_names[:1]:
    try:
        # Construct file paths
        zufang_path = os.path.join(zufang_base_path, f'{district}_zufang_data.csv')
        chengjiao_path = os.path.join(chengjiao_base_path, f'{district}_chengjiao_data.csv')
        
        # Read the data files
        df_zufang = process_zufang_data(pd.read_csv(zufang_path))
        df_chengjiao = process_chengjiao_data(pd.read_csv(chengjiao_path))
        
        # (Optional) Add any processing steps here

    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
    except pd.errors.EmptyDataError as e:
        print(f"Error: No data - {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [9]:
# Calculate annual rent in df_zufang
df_zufang['annual_rent'] = df_zufang['price'] * 12 /10000

# Standardizing or cleaning community names if necessary
# df_zufang['community'] = df_zufang['community'].str.lower().str.strip()
# df_chengjiao['community'] = df_chengjiao['community'].str.lower().str.strip()

# Merging the DataFrames on 'community'
combined_df = pd.merge(df_zufang, df_chengjiao, on=['community','total_room'], suffixes=('_rent', '_sale'))

# Calculating the rental-to-sale ratio
combined_df['rental_to_sale_ratio'] = combined_df['annual_rent'] / combined_df['total_price']

In [10]:
combined_df[['community', 'total_room','deal_date','annual_rent', 'total_price', 'rental_to_sale_ratio']]

,community,total_room,deal_date,annual_rent,total_price,rental_to_sale_ratio
0,百乐小区,2室1厅,2022.09.16,11.6892,539.8,0.021655
1,百乐小区,2室1厅,2022.09.16,11.6892,539.8,0.021655
2,中山北路805弄,2室1厅,2024.03.12,7.2000,270.0,0.026667
3,中山北路805弄,2室1厅,2023.09.04,7.2000,338.0,0.021302
4,中山北路805弄,2室1厅,2023.06.05,7.2000,400.0,0.018000
...,...,...,...,...,...,...
75950,延长中路451弄,2室1厅,2016.07.30,6.3000,288.0,0.021875
75951,升平小区,2室1厅,2023.01.14,8.8200,488.0,0.018074
75952,升平小区,2室1厅,2021.01.08,8.8200,565.0,0.015611
75953,升平小区,2室1厅,2021.01.02,8.8200,561.0,0.015722


# 成交

In [44]:
chengjiao_list = []
for file_ in glob(r'/Users/toshiro/repos/LianJia/chengjiao/data/*.csv'):
    df_ = pd.read_csv(file_)
    chengjiao_list.append(df_)

df_chengjiao = process_chengjiao_data(pd.concat(chengjiao_list))

In [45]:
df_chengjiao['deal_date'] = pd.to_datetime(df_chengjiao['deal_date']).dt.strftime('%Y.%m.%d')

In [46]:
# Extract year from deal_date
df_chengjiao['year'] = pd.to_datetime(df_chengjiao['deal_date']).dt.year

# Group by total_room and year, then calculate mean of transaction_day
average_transaction_days = df_chengjiao.groupby(['total_room', 'year'])['transaction_day'].mean().reset_index()

In [56]:
df_chengjiao.loc[df_chengjiao['area'] == '1室1厅 33.64']

,district,title,link,orientation,deal_date,total_price,position,unit_price,features,list_amount,transaction_day,agent_name,agent_link,community,total_room,area,year
5959,Huangpu,丽文小区 1室1厅 33.64平米,https://sh.lianjia.com/chengjiao/107108198674....,南 | 精装,2023.12.16,293.0,中楼层(共7层) 1985年板楼,87099.0,"房屋满五年, 近地铁",313.0,71.0,张晶晶,https://dianpu.lianjia.com/1000000022094948/,丽文小区,,1室1厅 33.64,2023


In [58]:
df_chengjiao.dropna(subset=['total_room'],inplace=True)

In [57]:
# Define bins and labels for the area ranges
bins = [0, 60, 80, 120, float('inf')]
labels = ['<60', '60-80', '80-120', '>120']

# Categorize area into ranges
# df_chengjiao.drop(index=[5912], axis=0,inplace=True)
df_chengjiao.dropna(subset=['total_room'],inplace=True)
df_chengjiao['area'] = df_chengjiao['area'].astype(float)
df_chengjiao['area_range'] = pd.cut(df_chengjiao['area'], bins=bins, labels=labels, right=False)

# Group by total_room, year, and area_range, then calculate mean of transaction_day
average_transaction_days_by_area = df_chengjiao.groupby(['total_room', 'year', 'area_range'])['transaction_day'].mean().reset_index()

ValueError: could not convert string to float: '1室1厅 33.64'

In [39]:
average_transaction_days_by_area.loc[average_transaction_days_by_area['year']>=2021].dropna(subset=['transaction_day'])

,total_room,year,area_range,transaction_day
208,1室0厅,2021,<60,94.155844
212,1室0厅,2022,<60,102.466019
216,1室0厅,2023,<60,104.470588
217,1室0厅,2023,60-80,199.000000
220,1室0厅,2024,<60,133.592593
...,...,...,...,...
2171,5房间3卫,2021,>120,316.333333
2175,5房间3卫,2022,>120,542.000000
2231,6室2厅,2022,>120,187.000000
2239,6室2厅,2024,>120,659.000000
